Zone d'importation

In [67]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, date
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

## Analyse exploratoire

    Dataset

In [2]:
# Chargement du dataset

data = pd.read_csv("Vente.csv", delimiter="\t")

In [3]:
# Copy du dataset

dataseg = data.copy()

In [4]:
#Cconvertir "Dt_Customer" en Datetime

dataseg["Dt_Customer"] = pd.to_datetime(dataseg["Dt_Customer"])
dataseg

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,2012-04-09,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,2014-08-03,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,2014-10-02,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,...,5,0,0,0,0,0,0,3,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,10870,1967,Graduation,Married,61223.0,0,1,2013-06-13,46,709,...,5,0,0,0,0,0,0,3,11,0
2236,4001,1946,PhD,Together,64014.0,2,1,2014-10-06,56,406,...,7,0,0,0,1,0,0,3,11,0
2237,7270,1981,Graduation,Divorced,56981.0,0,0,2014-01-25,91,908,...,6,0,1,0,0,0,0,3,11,0
2238,8235,1956,Master,Together,69245.0,0,1,2014-01-24,8,428,...,3,0,0,0,0,0,0,3,11,0


Etude partielle des Données du dataset

In [5]:
dataseg.columns

Index(['ID', 'Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome',
       'Teenhome', 'Dt_Customer', 'Recency', 'MntWines', 'MntFruits',
       'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
       'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
       'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
       'AcceptedCmp2', 'Complain', 'Z_CostContact', 'Z_Revenue', 'Response'],
      dtype='object')

In [6]:
dataseg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID                   2240 non-null   int64         
 1   Year_Birth           2240 non-null   int64         
 2   Education            2240 non-null   object        
 3   Marital_Status       2240 non-null   object        
 4   Income               2216 non-null   float64       
 5   Kidhome              2240 non-null   int64         
 6   Teenhome             2240 non-null   int64         
 7   Dt_Customer          2240 non-null   datetime64[ns]
 8   Recency              2240 non-null   int64         
 9   MntWines             2240 non-null   int64         
 10  MntFruits            2240 non-null   int64         
 11  MntMeatProducts      2240 non-null   int64         
 12  MntFishProducts      2240 non-null   int64         
 13  MntSweetProducts     2240 non-nul

In [7]:
#dataseg.describe(include='all'/[object],['category'], datetime_is_numeric=True)   Pour tout numérique, catégorie ou objet

dataseg.describe(datetime_is_numeric=True) # Que pour les variables quantitatives et la date

,ID,Year_Birth,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
count,2240.000000,2240.000000,2216.000000,2240.000000,2240.000000,2240,2240.000000,2240.000000,2240.000000,2240.000000,...,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.0,2240.0,2240.000000
mean,5592.159821,1968.805804,52247.251354,0.444196,0.506250,2013-07-11 22:57:38.571432192,49.109375,303.935714,26.302232,166.950000,...,5.316518,0.072768,0.074554,0.072768,0.064286,0.013393,0.009375,3.0,11.0,0.149107
min,0.000000,1893.000000,1730.000000,0.000000,0.000000,2012-01-08 00:00:00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,11.0,0.000000
25%,2828.250000,1959.000000,35303.000000,0.000000,0.000000,2013-01-19 18:00:00,24.000000,23.750000,1.000000,16.000000,...,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,11.0,0.000000
50%,5458.500000,1970.000000,51381.500000,0.000000,0.000000,2013-07-11 00:00:00,49.000000,173.500000,8.000000,67.000000,...,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,11.0,0.000000
75%,8427.750000,1977.000000,68522.000000,1.000000,1.000000,2013-12-30 06:00:00,74.000000,504.250000,33.000000,232.000000,...,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,11.0,0.000000
max,11191.000000,1996.000000,666666.000000,2.000000,2.000000,2014-12-06 00:00:00,99.000000,1493.000000,199.000000,1725.000000,...,20.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.0,11.0,1.000000
std,3246.662198,11.984069,25173.076661,0.538398,0.544538,NaN,28.962453,336.597393,39.773434,225.715373,...,2.426645,0.259813,0.262728,0.259813,0.245316,0.114976,0.096391,0.0,0.0,0.356274


In [8]:
dataseg.describe(include=[object]) # Que pour les variables qualitative

,Education,Marital_Status
count,2240,2240
unique,5,8
top,Graduation,Married
freq,1127,864


In [9]:
# Les différentes montants dépensés par produits que propose l'entreprise

depense = dataseg[["MntWines","MntFruits","MntMeatProducts","MntFishProducts","MntSweetProducts","MntGoldProds"]]
depense.head(10)

,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds
0,635,88,546,172,88,88
1,11,1,6,2,1,6
2,426,49,127,111,21,42
3,11,4,20,10,3,5
4,173,43,118,46,27,15
5,520,42,98,0,42,14
6,235,65,164,50,49,27
7,76,10,56,3,1,23
8,14,0,24,3,3,2
9,28,0,6,1,1,13


In [10]:
depense.describe()

,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds
count,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000
mean,303.935714,26.302232,166.950000,37.525446,27.062946,44.021875
std,336.597393,39.773434,225.715373,54.628979,41.280498,52.167439
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,23.750000,1.000000,16.000000,3.000000,1.000000,9.000000
50%,173.500000,8.000000,67.000000,12.000000,8.000000,24.000000
75%,504.250000,33.000000,232.000000,50.000000,33.000000,56.000000
max,1493.000000,199.000000,1725.000000,259.000000,263.000000,362.000000


In [11]:
# Acceptation d'offre par campagne 
# Nous donne une idée de la campagne qui a atteint la satisfaction du client

campagne = dataseg[["AcceptedCmp1", "AcceptedCmp2", "AcceptedCmp3", "AcceptedCmp4", "AcceptedCmp5", "Response"]]
campagne.head(10)

,AcceptedCmp1,AcceptedCmp2,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,Response
0,0,0,0,0,0,1
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
5,0,0,0,0,0,0
6,0,0,0,0,0,0
7,0,0,0,0,0,0
8,0,0,0,0,0,1
9,0,0,1,0,0,0


In [12]:
# Lieu où les clients achètent les différents produits

site_achat = dataseg[["NumWebPurchases", "NumCatalogPurchases", "NumStorePurchases"]]
site_achat.head(10)

,NumWebPurchases,NumCatalogPurchases,NumStorePurchases
0,8,10,4
1,1,1,2
2,8,2,10
3,2,0,4
4,5,3,6
5,6,4,10
6,7,3,7
7,4,0,4
8,3,0,2
9,1,0,0


In [13]:
site_achat.describe()

,NumWebPurchases,NumCatalogPurchases,NumStorePurchases
count,2240.000000,2240.000000,2240.000000
mean,4.084821,2.662054,5.790179
std,2.778714,2.923101,3.250958
min,0.000000,0.000000,0.000000
25%,2.000000,0.000000,3.000000
50%,4.000000,2.000000,5.000000
75%,6.000000,4.000000,8.000000
max,27.000000,28.000000,13.000000


In [14]:
# Fonction pour récupérer le reste des colonnes

def dropcol (dataframe):
    for col in depense & campagne & site_achat:
        dataframe = dataframe.drop([col], axis=1)
    return dataframe

In [15]:
# Infos perso du client

perso = dropcol(dataseg)
perso.head(10)

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,NumDealsPurchases,NumWebVisitsMonth,Complain,Z_CostContact,Z_Revenue
0,5524,1957,Graduation,Single,58138.0,0,0,2012-04-09,58,3,7,0,3,11
1,2174,1954,Graduation,Single,46344.0,1,1,2014-08-03,38,2,5,0,3,11
2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,1,4,0,3,11
3,6182,1984,Graduation,Together,26646.0,1,0,2014-10-02,26,2,6,0,3,11
4,5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,5,5,0,3,11
5,7446,1967,Master,Together,62513.0,0,1,2013-09-09,16,2,6,0,3,11
6,965,1971,Graduation,Divorced,55635.0,0,1,2012-11-13,34,4,6,0,3,11
7,6177,1985,PhD,Married,33454.0,1,0,2013-08-05,32,2,8,0,3,11
8,4855,1974,PhD,Together,30351.0,1,0,2013-06-06,19,1,9,0,3,11
9,5899,1950,PhD,Together,5648.0,1,1,2014-03-13,68,1,20,0,3,11


Vérification des valeurs manqauntes

In [16]:
# Nous n'avons que 24 valeurs manquants dans la colonne "Income", représentant 1% ce qui pourrait être ignoré.

dataseg.isnull().sum()/2240

ID                     0.000000
Year_Birth             0.000000
Education              0.000000
Marital_Status         0.000000
Income                 0.010714
Kidhome                0.000000
Teenhome               0.000000
Dt_Customer            0.000000
Recency                0.000000
MntWines               0.000000
MntFruits              0.000000
MntMeatProducts        0.000000
MntFishProducts        0.000000
MntSweetProducts       0.000000
MntGoldProds           0.000000
NumDealsPurchases      0.000000
NumWebPurchases        0.000000
NumCatalogPurchases    0.000000
NumStorePurchases      0.000000
NumWebVisitsMonth      0.000000
AcceptedCmp3           0.000000
AcceptedCmp4           0.000000
AcceptedCmp5           0.000000
AcceptedCmp1           0.000000
AcceptedCmp2           0.000000
Complain               0.000000
Z_CostContact          0.000000
Z_Revenue              0.000000
Response               0.000000
dtype: float64

In [17]:
# Voir les différentes modalités et portion

dataseg["Education"].value_counts(normalize=True)

Graduation    0.503125
PhD           0.216964
Master        0.165179
2n Cycle      0.090625
Basic         0.024107
Name: Education, dtype: float64

In [18]:
# Avoir un aperçu des différentes modalités, mais franchement je ne sais pas vraiment la définition de certaine modalités.

dataseg["Marital_Status"].value_counts(normalize=True)

Married     0.385714
Together    0.258929
Single      0.214286
Divorced    0.103571
Widow       0.034375
Alone       0.001339
YOLO        0.000893
Absurd      0.000893
Name: Marital_Status, dtype: float64

In [19]:
# Cette colonne n'a que des "3", ce n'est pas vraiment informative

dataseg["Z_CostContact"].value_counts(normalize=True)

3    1.0
Name: Z_CostContact, dtype: float64

In [20]:
# Cette colonne n'a que des "11", pareil que "Z_CostContact"

dataseg["Z_Revenue"].value_counts(normalize=True)

11    1.0
Name: Z_Revenue, dtype: float64

Nous disposons de deux variables qui ne figurent pas dans le descriptif entre autres "Z_CostContact" et "Z_Revenue".
Leurs valeurs sont fixes resp à "3" et "11". 
Tous les individus ont les mêmes infos cela ne nous avance pas à grand chose. Elles pourront être supprimées dans la suite.

. La colonne "Dt_Customer" qui est la Date d'inscription peut être supprimée dans la suite elle aussi car très peu informative.
<br> Je ne m'intéresse pas à cette date car j'ai choisi la méthode de segmentation a postériori ou cluster-based segmentation. Avec cette méthode on ne part pas de critères ou de règles prédéfinies mais on observe les similitudes entre des visiteurs et on les regroupe ensuite en fonction de celles-ci.
<br> . La colonne "ID" de même.

In [21]:
# Trouver l'âge des clients

def convert_age (nombre):
    today = datetime.now().date() # syntaxe pour avoir la date d'
    year = today.strftime("%Y")
    age = int(year) - nombre
    return age

In [22]:
# Ajoute d'une nouvelle colonne "Age".

dataseg["Age"] = dataseg["Year_Birth"].map(convert_age)
dataseg["Age"]

0       64
1       67
2       56
3       37
4       40
        ..
2235    54
2236    75
2237    40
2238    65
2239    67
Name: Age, Length: 2240, dtype: int64

In [23]:
dataseg["Age"].describe()

count    2240.000000
mean       52.194196
std        11.984069
min        25.000000
25%        44.000000
50%        51.000000
75%        62.000000
max       128.000000
Name: Age, dtype: float64

In [24]:
# Mise à jour du dataset

dataseg

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response,Age
0,5524,1957,Graduation,Single,58138.0,0,0,2012-04-09,58,635,...,0,0,0,0,0,0,3,11,1,64
1,2174,1954,Graduation,Single,46344.0,1,1,2014-08-03,38,11,...,0,0,0,0,0,0,3,11,0,67
2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,...,0,0,0,0,0,0,3,11,0,56
3,6182,1984,Graduation,Together,26646.0,1,0,2014-10-02,26,11,...,0,0,0,0,0,0,3,11,0,37
4,5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,...,0,0,0,0,0,0,3,11,0,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,10870,1967,Graduation,Married,61223.0,0,1,2013-06-13,46,709,...,0,0,0,0,0,0,3,11,0,54
2236,4001,1946,PhD,Together,64014.0,2,1,2014-10-06,56,406,...,0,0,0,1,0,0,3,11,0,75
2237,7270,1981,Graduation,Divorced,56981.0,0,0,2014-01-25,91,908,...,0,1,0,0,0,0,3,11,0,40
2238,8235,1956,Master,Together,69245.0,0,1,2014-01-24,8,428,...,0,0,0,0,0,0,3,11,0,65


Visualisation

In [25]:
depense.columns

Index(['MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts',
       'MntSweetProducts', 'MntGoldProds'],
      dtype='object')

In [ ]:

for col in perso.columns : # variable continue
    plt.figure(figsize=(12,8))
    # affichage des variable continue
    sns.countplot(x = col, hue="MntWines",data = data)
    plt.legend()
    plt.show()

In [ ]:
for col in depense : # variable continue
    plt.figure(figsize=(20,10))
    sns.distplot(dataseg[col]) # affichage des variable continue
    plt.show()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(data=(depense & perso).corr().round(1), annot=True)
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(data=campagne.corr().round(1), annot=True)
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(data=site_achat.corr().round(1), annot=True)
plt.show()

In [ ]:
for col in data.columns : # variable continue
    plt.figure(figsize=(12,8))
    # affichage des variable continue
    sns.countplot(x = col, hue="Class",data = data)
    plt.legend()
    plt.show()

## Preprocessing

In [101]:
dataseg

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response,Age
0,5524,1957,4,5,58138.0,0,0,2012-04-09,58,635,...,0,0,0,0,0,0,3,11,1,64
1,2174,1954,4,5,46344.0,1,1,2014-08-03,38,11,...,0,0,0,0,0,0,3,11,0,67
2,4141,1965,4,6,71613.0,0,0,2013-08-21,26,426,...,0,0,0,0,0,0,3,11,0,56
3,6182,1984,4,6,26646.0,1,0,2014-10-02,26,11,...,0,0,0,0,0,0,3,11,0,37
4,5324,1981,3,7,58293.0,1,0,2014-01-19,94,173,...,0,0,0,0,0,0,3,11,0,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,10870,1967,4,7,61223.0,0,1,2013-06-13,46,709,...,0,0,0,0,0,0,3,11,0,54
2236,4001,1946,3,6,64014.0,2,1,2014-10-06,56,406,...,0,0,0,1,0,0,3,11,0,75
2237,7270,1981,4,4,56981.0,0,0,2014-01-25,91,908,...,0,1,0,0,0,0,3,11,0,40
2238,8235,1956,2,6,69245.0,0,1,2014-01-24,8,428,...,0,0,0,0,0,0,3,11,0,65


In [102]:
# Encodage

def encodage(dataframe):
    code = {
        'Graduation':4, 'PhD':3, 'Master':2, "2n Cycle":1, "Basic":0,
        'YOLO':0,'Absurd':1,'Alone':2,'Widow':3, 'Divorced':4,'Single':5, 'Together':6, 'Married':7,
}
    
    for col in dataframe.select_dtypes('object').columns:
        dataframe.loc[:,col] = dataframe[col].map(code)
    
    return dataframe

In [103]:
process_data = encodage(dataseg)

In [104]:
# Suppression des colonnes que j'avais trouvé superflux.

process_data = process_data.drop(["ID","Dt_Customer","Z_CostContact", "Z_Revenue","Year_Birth"], axis=1)

In [105]:
# Les données manquantes sont très peu alors je supprime la ligne des 24...
process_data = process_data.dropna()

In [106]:
process_data

,Education,Marital_Status,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,...,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,Age
0,4,5,58138.0,0,0,58,635,88,546,172,...,4,7,0,0,0,0,0,0,1,64
1,4,5,46344.0,1,1,38,11,1,6,2,...,2,5,0,0,0,0,0,0,0,67
2,4,6,71613.0,0,0,26,426,49,127,111,...,10,4,0,0,0,0,0,0,0,56
3,4,6,26646.0,1,0,26,11,4,20,10,...,4,6,0,0,0,0,0,0,0,37
4,3,7,58293.0,1,0,94,173,43,118,46,...,6,5,0,0,0,0,0,0,0,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,4,7,61223.0,0,1,46,709,43,182,42,...,4,5,0,0,0,0,0,0,0,54
2236,3,6,64014.0,2,1,56,406,0,30,0,...,5,7,0,0,0,1,0,0,0,75
2237,4,4,56981.0,0,0,91,908,48,217,32,...,13,6,0,1,0,0,0,0,0,40
2238,2,6,69245.0,0,1,8,428,30,214,80,...,10,3,0,0,0,0,0,0,0,65


In [99]:
# Normalisons les variables

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(process_data)

In [115]:
process_data = pd.DataFrame(X_scaled, index=process_data.index, columns=process_data.columns)

In [116]:
process_data.isna().sum()

Education              0
Marital_Status         0
Income                 0
Kidhome                0
Teenhome               0
Recency                0
MntWines               0
MntFruits              0
MntMeatProducts        0
MntFishProducts        0
MntSweetProducts       0
MntGoldProds           0
NumDealsPurchases      0
NumWebPurchases        0
NumCatalogPurchases    0
NumStorePurchases      0
NumWebVisitsMonth      0
AcceptedCmp3           0
AcceptedCmp4           0
AcceptedCmp5           0
AcceptedCmp1           0
AcceptedCmp2           0
Complain               0
Response               0
Age                    0
dtype: int64

In [124]:
X = process_data
X.head()

,Education,Marital_Status,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,...,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,Age
0,1.00,0.714286,0.084832,0.0,0.0,0.585859,0.425318,0.442211,0.316522,0.664093,...,0.307692,0.35,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.378641
1,1.00,0.714286,0.067095,0.5,0.5,0.383838,0.007368,0.005025,0.003478,0.007722,...,0.153846,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.407767
2,1.00,0.857143,0.105097,0.0,0.0,0.262626,0.285332,0.246231,0.073623,0.428571,...,0.769231,0.20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.300971
3,1.00,0.857143,0.037471,0.5,0.0,0.262626,0.007368,0.020101,0.011594,0.038610,...,0.307692,0.30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.116505
4,0.75,1.000000,0.085065,0.5,0.0,0.949495,0.115874,0.216080,0.068406,0.177606,...,0.461538,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.145631


## Modélisation 

J'ai en tête d'utiliser le CLUSTERING pour la suite...

In [88]:
# Zone importation

sns.set_style('darkgrid')

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier

In [126]:
kmeans = KMeans(n_clusters=3)
kmeans.fit(process_data)

cluster_labels = kmeans.labels_
cluster_labels

array([0, 1, 2, ..., 2, 2, 0])

array([1, 0, 2, ..., 2, 2, 0])